In [7]:
import os
import dask.dataframe as dd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


# Specify dtypes
dtype = {'CRSElapsedTime': 'float64', 'TailNum': 'object'}
year = str(input('Enter the year 1990-1999: '))
# Read the data with specified dtypes
df = dd.read_csv(os.path.join('data', 'nycflights', f'{year}.csv'), 
                 parse_dates={'Date': [0, 1, 2]}, dtype=dtype)
df.head()

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
# 1. Departure Delays from JFK Airport
jfk_departure_delays = df[df['Origin'] == 'JFK']['DepDelay'].compute()

# Visualization of Departure Delays from JFK Airport
plt.figure(figsize=(10, 6))
sns.histplot(data=jfk_departure_delays.dropna(), bins=50)
plt.title("Histogram of JFK airport departure delays")
plt.xlabel("Mins")
plt.ylabel("Frequency")
plt.show()

In [ ]:
# 2. Choosing Airlines
airline_delay_avg = df.groupby('UniqueCarrier')['DepDelay'].mean().compute()
best_airline_jfk = airline_delay_avg.idxmin()

# Group by carrier and calculate mean and standard deviation of departure delay
jfk_carrier = df[df['Origin'] == 'JFK'].groupby('UniqueCarrier')['DepDelay'].agg(['mean', 'std']).compute()

# Calculate median separately
jfk_carrier['median'] = df[df['Origin'] == 'JFK'].groupby('UniqueCarrier')['DepDelay'].apply(lambda x: x.quantile(0.5)).compute()

# Rename columns
jfk_carrier.columns = ['mean_delay', 'std_dev_delay', 'median_delay']

# Sort by mean delay in descending order
jfk_carrier = jfk_carrier.sort_values(by='mean_delay', ascending=False)

# Calculate flight frequency for each carrier
jfk_flight_freq = df[df['Origin'] == 'JFK']['UniqueCarrier'].value_counts().compute().reset_index()

# Rename columns
jfk_flight_freq.columns = ['carrier', 'freq']

# Merge the two dataframes
merged_df = pd.merge(jfk_carrier, jfk_flight_freq, left_index=True, right_on='carrier')

# Sort by mean delay in descending order
merged_df = merged_df.sort_values(by='mean_delay', ascending=False)

print("\n2. Best Airline from JFK (Least Average Departure Delay):", best_airline_jfk)
#Print the merged dataframe
merged_df



In [ ]:
# 3. Best Time to Fly Out of JFK
jfk_departure_times = df[df['Origin'] == 'JFK'][['DepTime', 'DepDelay']]
# Extract hour from Departure Time
jfk_departure_times['Hour'] = jfk_departure_times['DepTime'].map(lambda x: x, meta=('DepTime', 'int'))
best_time_jfk = jfk_departure_times.groupby('Hour')['DepDelay'].mean().idxmin().compute()

# Convert 'Date' column to datetime and extract month
df['Month'] = df['Date'].dt.month

# Now you can group by 'Month' and calculate mean, standard deviation, median, and interquartile range of departure delay
jfk_origin = df[df['Origin'] == 'JFK'].groupby('Month')['DepDelay'].agg(['mean', 'std', 'count']).compute()

# Reset index
jfk_origin = jfk_origin.reset_index()

# Calculate median separately
jfk_origin['median'] = df[df['Origin'] == 'JFK'].groupby('Month')['DepDelay'].apply(lambda x: x.quantile(0.5)).compute()

# Calculate interquartile range separately
jfk_origin['iqr'] = df[df['Origin'] == 'JFK'].groupby('Month')['DepDelay'].apply(lambda x: x.quantile(0.75) - x.quantile(0.25)).compute()

# Rename columns
jfk_origin.columns = ['Month', 'mean_delay', 'std_dev_delay', 'frequency', 'median_delay', 'iqr']

# Sort by mean delay in descending order
jfk_origin = jfk_origin.sort_values(by='mean_delay', ascending=False)

# Print the dataframe
print(jfk_origin)

# Convert 'Month' to categorical type for proper sorting and plotting
jfk_origin['Month'] = pd.Categorical(jfk_origin['Month'], categories=range(1, 13), ordered=True)

# Create a boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='Month', y='mean_delay', data=jfk_origin)

plt.title('Boxplot of Mean Departure Delays by Month')
plt.xlabel('Month')
plt.ylabel('Mean Departure Delay')
plt.show()


In [ ]:

# 4. Airport Punctuality
airport_punctuality = df.groupby('Origin')[['ArrDelay', 'DepDelay']].mean().compute()

# Add 'dep_type' column
df['dep_type'] = df['DepDelay'].apply(lambda x: 'On Time' if x < 5 else 'Delayed')

# Drop rows with missing values
df = df.dropna()

# Calculate on-time departure rate for each origin
ot_dep_rate = df.groupby('Origin')['dep_type'].apply(lambda x: (x == 'On Time').sum() / len(x)).compute()

# Sort by on-time departure rate in descending order
ot_dep_rate = ot_dep_rate.sort_values(ascending=False)

print("\n4. Airport Punctuality (Mean Arrival and Departure Delays):")
print(airport_punctuality)

# Print the result
print(ot_dep_rate)


In [ ]:
# Transform the data
df_transformed = df.groupby(['Origin', 'dep_type']).size().reset_index().rename(columns={0: 'count'})

# Convert Dask DataFrame to Pandas DataFrame
df_transformed = df_transformed.compute()

# Pivot the data
df_pivot = df_transformed.pivot(index='Origin', columns='dep_type', values='count').fillna(0)

# Create a stacked bar plot
df_pivot.plot(kind='bar', stacked=True, figsize=(10, 6))

plt.title('Number of Flights for Each Origin by Departure Type')
plt.xlabel('Origin')
plt.ylabel('Number of Flights')
# plt.show()

In [ ]:
# Drop rows with missing values
df = df.dropna()

# Calculate number of flights for each origin
no_of_flights = df.groupby('Origin').size().compute()

# Sort by number of flights in descending order
no_of_flights = no_of_flights.sort_values(ascending=False)

# Print the result
print(no_of_flights)


In [ ]:
# Add 'arr_type' column
df['arr_type'] = df['ArrDelay'].apply(lambda x: 'On Time' if x <= 0 else 'Delayed')

# Drop rows with missing values
df = df.dropna()

# Calculate on-time arrival rate for each origin
arr_rate = df.groupby('Origin')['arr_type'].apply(lambda x: (x == 'On Time').sum() / len(x)).compute()

# Sort by on-time arrival rate in descending order
arr_rate = arr_rate.sort_values(ascending=False)

# Print the result
print(arr_rate)

In [ ]:
#what carrier travels the fastest
# Convert air time to hours and create a new column 'air_time_hour'
df['air_time_hour'] = df['AirTime'].map_partitions(lambda x: round(x / 60, 1))

# Create a new column 'average_speed' by dividing distance by air_time_hour
df = df.dropna()
df['average_speed'] = df['Distance'] / df['air_time_hour']

# Sort by average speed in descending order and select the 'average_speed', 'TailNum', and 'UniqueCarrier' columns
df_sorted = df.sort_values(by='average_speed', ascending=False)[['average_speed', 'TailNum', 'UniqueCarrier']]

# Print the result
print(df_sorted.compute())

In [ ]:
# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(df['average_speed'].compute(), df['Distance'].compute())

plt.title('Distance vs Average Speed')
plt.xlabel('Average Speed')
plt.ylabel('Distance')
plt.show()

In [ ]:
# 5. Carrier Handling Most Long Haul Flights
long_haul_flights = df[df['Distance'] > 3000]  # Assuming long haul as > 3000 miles
carrier_long_haul = long_haul_flights['UniqueCarrier'].value_counts().idxmax().compute()

####################################################################################
# Filter flights with distance > 1000
df_filtered = df[df['Distance'] > 1000]

# Group by carrier and calculate frequency and mean distance
carrier_stats = df_filtered.groupby('UniqueCarrier').agg({'UniqueCarrier': 'count', 'Distance': 'mean'}).compute()

# Rename the columns
carrier_stats.columns = ['freq', 'mean_distance']

# Sort by frequency in descending order
carrier_stats = carrier_stats.sort_values(by='freq', ascending=False)

print("\n5. Carrier Handling Most Long Haul Flights:", carrier_long_haul)
# Print the result
print(carrier_stats)


In [ ]:
# 6. Popular Destination for Each Airport
# Group by origin and destination and calculate number of flights
popular_destinations = df.groupby(['Origin', 'Dest']).size().compute()

# Reset the index and rename the column
popular_destinations = popular_destinations.reset_index().rename(columns={0: 'n_flights'})

# Sort by number of flights in descending order and select the top 10
popular_destinations = popular_destinations.sort_values(by='n_flights', ascending=False).head(10)
print("\n6. Popular Destination for Each Airport:")
print(popular_destinations)